In [1]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
import torch
sys.path.append('./')  # uncomment for local import
import tangram as tg

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

tg.__version__

'1.0.4'

In [3]:
# Load metadata
info = pd.read_csv('/Users/tili/Desktop/CIIR/data/ST-AR_Visium_samples.csv')
info

,Patient,Patientid,Tumor area,Visium ID,Visum h5ad ID,name,type,width,height
0,TD-2532-BCSA1,BCSA1,TumA1,V19T26-012_A1,200116_ST-AR_BC_A_B_V19T26-012_KT_V1-Spot00000...,a1.012,TNBC,29184,29696
1,TD-2532-BCSA1,BCSA1,TumA2,V19T26-012_B1,200116_ST-AR_BC_A_B_V19T26-012_KT_V2-Spot00000...,b1.012,TNBC,29184,29696
2,TD-2532-BCSA1,BCSA1,TumB1,V19T26-012_C1,200116_ST-AR_BC_A_B_V19T26-012_KT_V3-Spot00000...,c1.012,TNBC,29184,29696
3,TD-2532-BCSA1,BCSA1,TumB2,V19T26-012_D1,200116_ST-AR_BC_A_B_V19T26-012_KT_V4-Spot00000...,d1.012,TNBC,29184,29696
4,TD-2532-BCSA2,BCSA2,TumB1,V10F24-112_C1,200616_BC-SA2_V10F24-112_KT.C1-Spot000001.h5ad,c1.112,HER2+,47616,48128
5,TD-2532-BCSA2,BCSA2,TumB2,V10F24-112_D1,200616_BC-SA2_V10F24-112_KT.D1-Spot000001.h5ad,d1.112,HER2+,47616,48128
6,TD-2532-BCSA2,BCSA2,TumC1,V10F24-113_A1,200715_BC_SA2_C_D_V10F24-113.A1-Spot000001.h5ad,a1.113,HER2+,48640,47104
7,TD-2532-BCSA2,BCSA2,TumC2,V10F24-113_B1,200715_BC_SA2_C_D_V10F24-113.B1-Spot000001.h5ad,b1.113,HER2+,48640,47104
8,TD-2532-BCSA2,BCSA2,TumD1,V10F24-113_C1,200715_BC_SA2_C_D_V10F24-113.C1-Spot000001.h5ad,c1.113,HER2+,48640,47104
9,TD-2532-BCSA2,BCSA2,TumD2,V10F24-113_D1,200715_BC_SA2_C_D_V10F24-113.D1-Spot000001.h5ad,d1.113,HER2+,48640,47104


In [4]:
# Get sample ID
info['sampleID'] = info['Patientid'] + info['Tumor area']
info

,Patient,Patientid,Tumor area,Visium ID,Visum h5ad ID,name,type,width,height,sampleID
0,TD-2532-BCSA1,BCSA1,TumA1,V19T26-012_A1,200116_ST-AR_BC_A_B_V19T26-012_KT_V1-Spot00000...,a1.012,TNBC,29184,29696,BCSA1TumA1
1,TD-2532-BCSA1,BCSA1,TumA2,V19T26-012_B1,200116_ST-AR_BC_A_B_V19T26-012_KT_V2-Spot00000...,b1.012,TNBC,29184,29696,BCSA1TumA2
2,TD-2532-BCSA1,BCSA1,TumB1,V19T26-012_C1,200116_ST-AR_BC_A_B_V19T26-012_KT_V3-Spot00000...,c1.012,TNBC,29184,29696,BCSA1TumB1
3,TD-2532-BCSA1,BCSA1,TumB2,V19T26-012_D1,200116_ST-AR_BC_A_B_V19T26-012_KT_V4-Spot00000...,d1.012,TNBC,29184,29696,BCSA1TumB2
4,TD-2532-BCSA2,BCSA2,TumB1,V10F24-112_C1,200616_BC-SA2_V10F24-112_KT.C1-Spot000001.h5ad,c1.112,HER2+,47616,48128,BCSA2TumB1
5,TD-2532-BCSA2,BCSA2,TumB2,V10F24-112_D1,200616_BC-SA2_V10F24-112_KT.D1-Spot000001.h5ad,d1.112,HER2+,47616,48128,BCSA2TumB2
6,TD-2532-BCSA2,BCSA2,TumC1,V10F24-113_A1,200715_BC_SA2_C_D_V10F24-113.A1-Spot000001.h5ad,a1.113,HER2+,48640,47104,BCSA2TumC1
7,TD-2532-BCSA2,BCSA2,TumC2,V10F24-113_B1,200715_BC_SA2_C_D_V10F24-113.B1-Spot000001.h5ad,b1.113,HER2+,48640,47104,BCSA2TumC2
8,TD-2532-BCSA2,BCSA2,TumD1,V10F24-113_C1,200715_BC_SA2_C_D_V10F24-113.C1-Spot000001.h5ad,c1.113,HER2+,48640,47104,BCSA2TumD1
9,TD-2532-BCSA2,BCSA2,TumD2,V10F24-113_D1,200715_BC_SA2_C_D_V10F24-113.D1-Spot000001.h5ad,d1.113,HER2+,48640,47104,BCSA2TumD2


In [6]:
# Map the ST data using marker genes. All datasets needs to be unzipped, 
# resulting h5ad and csv files should be placed in the data folder

for row in info.iterrows():
    id = row[1]['sampleID']
    # Load ST data
    path = '/Users/tili/Desktop/CIIR/data/ST_data/Tangram/'+id+".h5ad"
    ad_sp = sc.read_h5ad(path)
    if row[1]['type'] == "TNBC":
        path = os.path.join('/Users/tili/Desktop/CIIR/data/Wu_etal_2021_BRCA_scRNASeq/Tangram/','tnbc.h5ad')
        ad_sc = sc.read_h5ad(path)
    elif row[1]['type'] == "HER2+":
        path = os.path.join('/Users/tili/Desktop/CIIR/data/Wu_etal_2021_BRCA_scRNASeq/Tangram/','her2.h5ad')
        ad_sc = sc.read_h5ad(path)
    
    # Usually, the data input should be in raw count, especially if the ST data are in raw count form
    np.unique(ad_sc.X.toarray()[0, :])

    # Here, we only do some light pre-processing as library size correction (in scanpy, via sc.pp.normalize) 
    # to normalize the number of count within each cell to a fixed number.

    # Sometimes, we apply more sophisticated pre-processing methods, for example for batch correction, 
    # although mapping works great with raw data.

    # Ideally, the single cell and spatial datasets, should exhibit signals as similar as possible and 
    # the pre-processing pipeline should be finalized to harmonize the signals
    #sc.pp.normalize_total(ad_sc)

    # Cell types are annotated in the subclass_label field, for which we plot cell counts.
    # Note that cell type proportion should be similar in the two datasets
    ad_sc.obs.celltype_minor.value_counts()
    
    # Tangram learns a spatial alignment of the single cell data so that the gene expression of the 
    # aligned single cell data is as similar as possible to that of the spatial data

    # Tangram only looks at a subset genes, specified by the user, called the training genes.
    # they need to bear interesting signals and to be measured with high quality.

    # Typically, a good start is to choose 100-1000 top marker genes, evenly stratified across cell types. 
    # Sometimes, we also use the entire transcriptome, 
    # or perform different mappings using different set of training genes to see how much the result change.

    df_genes = pd.read_csv('/Users/tili/Desktop/CIIR/data/ST_data/Tangram/features.csv', index_col=0)
    markers = np.reshape(df_genes.values, (-1, ))
    markers = list(markers)
    print(len(markers))
    
    # We now need to prepare the datasets for mapping by creating training_genes 
    # field in uns dictionary of the two AnnData structures.

    # This training_genes field contains genes subset on the list of training genes. 
    # This field will be used later inside the mapping function to create training datasets.

    # Also, the gene order needs to be the same in the datasets. This is because Tangram maps 
    # using only gene expression, so the j-th column in each matrix must correspond to the same gene.

    # And if data entries of a gene are all zero, this gene will be removed.

    # In the pp_adatas function, the gene names are converted to lower case to get rid of the inconsistent capitalization. 
    # If this is not wanted, you can set the parameter gene_to_lowercase = False
    tg.pp_adatas(ad_sc, ad_sp, genes=markers)
    
    # the marker genes need to be shared by both dataset. If a gene is missing, pp_adatas will just take it out.
    # Finally, the assert line below is a good way to ensure that the genes in the training_genes field in uns 
    # are actually ordered in both AnnDatas.
    assert ad_sc.uns['training_genes'] == ad_sp.uns['training_genes']
    
    # Mapping should be interrupted after the score plateaus,
    # which can be controlled by passing the num_epochs parameter.

    # The score measures the similarity between the gene expression of the mapped cells 
    # vs spatial data: higher score means better mapping.

    # Note that we obtained excellent mapping even if Tangram converges to a low scores (the typical case is 
    # when the spatial data are very sparse): we use the score merely to assess convergence

    # If you are running Tangram with a GPU, uncomment device=cuda:0 and comment the line device=cpu.

    # For this basic mapping, we do not use regularizers. 
    # More sophisticated loss functions can be used using the Tangram library.
    # For example, you can pass your density_prior with the hyperparameter lambda_d to regularize the spatial 
    # density of cells. Currently uniform, rna_count_based and 
    # customized input array are supported for density_prior argument.

    # Instead of mapping single cells, we can "average" the cells within a cluster and map the averaged cells instead, 
    # which drammatically improves performances. This suggestion was proposed by Sten Linnarsson. 
    # To activate this mode, select mode='clusters' and pass the annotation field to cluster_label.
    ad_map = tg.map_cells_to_space(
        adata_sc=ad_sc,
        adata_sp=ad_sp,
        device='cpu',
        #mode='cells', # defualt mode 
        #density_prior='rna_count_based', #default method
        #device='cuda:0',
    )
    
    # The mapping results are stored in the returned AnnData structure, saved as ad_map, structured as following:
    # The cell-by-spot matrix X contains the probability of cell i to be in spot j.
    # The obs dataframe contains the metadata of the single cells.
    # The var dataframe contains the metadata of the spatial data.
    # The uns dictionary contains a dataframe with various information about the training genes 
    # (saved ad train_genes_df).

    # The most common application for mapping single cell data onto space is to transfer the cell 
    # type annotations onto space, dona via plot_cell_annotation, which visualizes spatial probability maps 
    # of the annotation in the obs dataframe (here, the subclass_label field). You can set robust argument to 
    # True and at the same time set the perc argument to set the range to the colormap, which would help remove outliers.

    ad_map
    tg.project_cell_annotations(ad_map, ad_sp, annotation='celltype_minor')

    # Save the results
    path = "/Users/tili/Desktop/CIIR/results/Tangram/Minor_HER2_genes/" + id + "_Tangram.csv"
    ad_sp.obsm["tangram_ct_pred"].to_csv(path, index = True)

5539


INFO:root:5262 training genes are saved in `uns``training_genes` of both single cell and spatial Anndatas.
INFO:root:15903 overlapped genes are saved in `uns``overlap_genes` of both single cell and spatial Anndatas.
INFO:root:uniform based density prior is calculated and saved in `obs``uniform_density` of the spatial Anndata.
INFO:root:rna count based density prior is calculated and saved in `obs``rna_count_based_density` of the spatial Anndata.
INFO:root:Allocate tensors for mapping.
INFO:root:Begin training with 5262 genes and rna_count_based density_prior in cells mode...
INFO:root:Printing scores every 100 epochs.


Score: 0.363, KL reg: 0.294
Score: 0.546, KL reg: 0.000
Score: 0.560, KL reg: 0.000
Score: 0.563, KL reg: 0.000
Score: 0.564, KL reg: 0.000
Score: 0.564, KL reg: 0.000
Score: 0.564, KL reg: 0.000
Score: 0.564, KL reg: 0.000
Score: 0.565, KL reg: 0.000
Score: 0.565, KL reg: 0.000


INFO:root:Saving results..
INFO:root:spatial prediction dataframe is saved in `obsm` `tangram_ct_pred` of the spatial AnnData.


5539


...

INFO:root:Saving results..
INFO:root:spatial prediction dataframe is saved in `obsm` `tangram_ct_pred` of the spatial AnnData.


5539


INFO:root:5262 training genes are saved in `uns``training_genes` of both single cell and spatial Anndatas.
INFO:root:15631 overlapped genes are saved in `uns``overlap_genes` of both single cell and spatial Anndatas.
INFO:root:uniform based density prior is calculated and saved in `obs``uniform_density` of the spatial Anndata.
INFO:root:rna count based density prior is calculated and saved in `obs``rna_count_based_density` of the spatial Anndata.
INFO:root:Allocate tensors for mapping.
INFO:root:Begin training with 5262 genes and rna_count_based density_prior in cells mode...
INFO:root:Printing scores every 100 epochs.


Score: 0.292, KL reg: 0.341
Score: 0.502, KL reg: 0.000
Score: 0.517, KL reg: 0.000
Score: 0.520, KL reg: 0.000
Score: 0.521, KL reg: 0.000
Score: 0.521, KL reg: 0.000
Score: 0.522, KL reg: 0.000
Score: 0.522, KL reg: 0.000
Score: 0.522, KL reg: 0.000
Score: 0.522, KL reg: 0.000


INFO:root:Saving results..
INFO:root:spatial prediction dataframe is saved in `obsm` `tangram_ct_pred` of the spatial AnnData.
